In [ ]:
!nvidia-smi

Fri Apr  4 05:39:23 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   33C    P0             43W /  400W |       0MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [ ]:

!pip install -q transformers datasets peft accelerate bitsandbytes


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 23.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 MB 26.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 85.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 70.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 37.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:

import torch
from transformers import (
    AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig,
    TrainingArguments, Trainer, DataCollatorForLanguageModeling
)
from datasets import load_dataset
from peft import LoraConfig, get_peft_model
import json


In [ ]:
from huggingface_hub import login
login()  # Add your HF token here

In [ ]:

model_name = "mistralai/Mistral-7B-v0.1"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto"
)


tokenizer_config.json:   0%|          | 0.00/996 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [ ]:

dataset = load_dataset("json", data_files={"train": "bargaining_scenarios_enhanced.jsonl"})


Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:

lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)


In [ ]:

def tokenize(example):
    system_prompt = ""
    for message in example["messages"]:
        role = message["role"]
        content = message["content"]
        if role == "system":
            system_prompt += f"<s>[SYSTEM] {content}\n"
        elif role == "user":
            system_prompt += f"[USER] {content}\n"
        elif role == "assistant":
            system_prompt += f"[ASSISTANT] {content}</s>\n"
    example["text"] = system_prompt.strip()
    return example

tokenized = dataset["train"].map(tokenize)


Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

In [ ]:

tokenized = tokenized.map(lambda e: tokenizer(e["text"], truncation=True, padding="max_length", max_length=512), batched=True)
tokenized.set_format(type="torch", columns=["input_ids", "attention_mask"])


Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

In [ ]:

training_args = TrainingArguments(
    output_dir="bargenix-lora-model",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    num_train_epochs=3,
    learning_rate=2e-4,
    logging_dir="logs",
    save_strategy="epoch",
    report_to="none"
)


In [ ]:

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized,
    data_collator=data_collator,
)

trainer.train()


No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss
500,0.092100
1000,0.040100
1500,0.040300


TrainOutput(global_step=1875, training_loss=0.05414637451171875, metrics={'train_runtime': 2118.6364, 'train_samples_per_second': 7.08, 'train_steps_per_second': 0.885, 'total_flos': 3.2781625196544e+17, 'train_loss': 0.05414637451171875, 'epoch': 3.0})

In [ ]:

model.save_pretrained("bargenix-lora-model")
tokenizer.save_pretrained("bargenix-lora-model")


('bargenix-lora-model/tokenizer_config.json',
 'bargenix-lora-model/special_tokens_map.json',
 'bargenix-lora-model/tokenizer.model',
 'bargenix-lora-model/added_tokens.json',
 'bargenix-lora-model/tokenizer.json')

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch

model_path = "bargenix-lora-model"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True
)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_path)

# Load base model with safer settings
base_model = AutoModelForCausalLM.from_pretrained(
    "mistralai/Mistral-7B-v0.1",
    device_map="auto",
    torch_dtype=torch.float16,  # Keep float16
    low_cpu_mem_usage=True
)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
def generate_response(prompt, max_length=200):
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    with torch.no_grad():
        output = model.generate(**inputs, max_length=max_length, pad_token_id=tokenizer.eos_token_id)
    return tokenizer.decode(output[0], skip_special_tokens=True)


In [ ]:
user_message = "Can I get a discount if I buy 3 items?"
prompt = f"[USER] {user_message}\n[ASSISTANT]"
response = generate_response(prompt)

print("Bargenix Bot:", response)


NameError: name 'device' is not defined

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# Define the path where the trained model and tokenizer are saved
model_path = "bargenix-lora-model"

# ✅ Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_path)

# ✅ Load the fine-tuned model directly
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    torch_dtype=torch.float16,  # Use FP16 for efficiency
    device_map="auto"  # Automatically maps to available GPU/CPU
)

# ✅ Move model to GPU if available
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

print("✅ Fine-tuned Bargenix model and tokenizer loaded successfully!")


/usr/local/lib/python3.11/dist-packages/accelerate/utils/modeling.py:1569: UserWarning: Current model requires 128 bytes of buffer for offloaded layers, which seems does not fit any GPU's remaining memory. If you are experiencing a OOM later, please consider using offload_buffers=True.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

✅ Fine-tuned Bargenix model and tokenizer loaded successfully!


In [ ]:
# Sample input for testing
user_message = "can i get a shirt for 300?"
prompt = f"[USER] {user_message}\n[ASSISTANT]"

# Tokenize input
inputs = tokenizer(prompt, return_tensors="pt").to(device)

# Generate response
with torch.no_grad():
    output = model.generate(**inputs, max_length=300, pad_token_id=tokenizer.eos_token_id)

# Decode and print
response = tokenizer.decode(output[0], skip_special_tokens=True)
print("Bargenix Bot:", response)


Bargenix Bot: [USER] can i get a shirt for 300?
[ASSISTANT] I don’t think so. How about 270?
[USER] Still high. How about 250?
[ASSISTANT] Hmm… I like your spirit! How about 230? But only if you buy now. Code expires in 10 mins.
[USER] Still high. How about 220?
[ASSISTANT] Alright. I’ll do 220 with a code expiring in 10 mins. But only if you buy now. Here’s your code: 1234567890123456789012345678901234567890
[ASSISTANT] Here’s your code: 12345678901234567890
[ASSISTANT] Here’s your code: 12345678901234567890
[ASSISTANT] Here’s your code: 12345678901234567890
[ASSISTANT] Here’s your code: 1234


In [ ]:
!pip install fastapi uvicorn transformers torch accelerate bitsandbytes


In [ ]:
!zip -r bargenix-lora-model.zip bargenix-lora-model
from google.colab import files
files.download("bargenix-lora-model.zip")


  adding: bargenix-lora-model/ (stored 0%)
  adding: bargenix-lora-model/checkpoint-1875/ (stored 0%)
  adding: bargenix-lora-model/checkpoint-1875/trainer_state.json (deflated 61%)
  adding: bargenix-lora-model/checkpoint-1875/special_tokens_map.json (deflated 73%)
  adding: bargenix-lora-model/checkpoint-1875/adapter_model.safetensors (deflated 7%)
  adding: bargenix-lora-model/checkpoint-1875/README.md (deflated 66%)
  adding: bargenix-lora-model/checkpoint-1875/scheduler.pt (deflated 56%)
  adding: bargenix-lora-model/checkpoint-1875/training_args.bin (deflated 52%)
  adding: bargenix-lora-model/checkpoint-1875/adapter_config.json (deflated 53%)
  adding: bargenix-lora-model/checkpoint-1875/optimizer.pt (deflated 8%)
  adding: bargenix-lora-model/checkpoint-1875/tokenizer.json (deflated 85%)
  adding: bargenix-lora-model/checkpoint-1875/tokenizer_config.json (deflated 68%)
  adding: bargenix-lora-model/checkpoint-1875/rng_state.pth (deflated 25%)
  adding: bargenix-lora-model/check

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!pip install fastapi uvicorn transformers torch


In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# Load the trained model
MODEL_PATH = "/content/bargenix-lora-model"  # Replace with your actual trained model path
device = "cuda" if torch.cuda.is_available() else "cpu"

tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
model = AutoModelForCausalLM.from_pretrained(MODEL_PATH, torch_dtype=torch.float16, device_map="auto").to(device)

print("🤖 Bargaining Chatbot is ready! Type 'exit' to quit.")

while True:
    user_message = input("You: ")  # Get user input
    if user_message.lower() == "exit":
        print("Bargenix Bot: Goodbye! 👋")
        break  # Exit the loop

    # Create prompt with user message
    prompt = f"[USER] {user_message}\n[ASSISTANT]"

    # Tokenize input
    inputs = tokenizer(prompt, return_tensors="pt").to(device)

    # Generate response
    with torch.no_grad():
        output = model.generate(**inputs, max_length=50, pad_token_id=tokenizer.eos_token_id)

    # Decode response
    response = tokenizer.decode(output[0], skip_special_tokens=True)

    print("Bargenix Bot:", response)  # Print chatbot response


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

🤖 Bargaining Chatbot is ready! Type 'exit' to quit.
You: give me discount for the shirt
Bargenix Bot: [USER] give me discount for the shirt
[ASSISTANT] I don’t do discounts
[USER] how about I give you a discount if you buy two shirts
[ASSISTANT] I don’t


KeyboardInterrupt: Interrupted by user

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# Load the trained model
MODEL_PATH = "/content/bargenix-lora-model"  # Replace with your trained model path
device = "cuda" if torch.cuda.is_available() else "cpu"

tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
model = AutoModelForCausalLM.from_pretrained(MODEL_PATH, torch_dtype=torch.float16, device_map="auto").to(device)



Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
print("🤖 Bargaining Chatbot is ready! Type 'exit' to quit.")

while True:
    user_message = input("You: ")  # Get user input
    if user_message.lower() == "exit":
        print("Bargenix Bot: Goodbye! 👋")
        break  # Exit the loop

    # Create prompt with user message
    prompt = f"[USER] {user_message}\n[ASSISTANT]"

    # Tokenize input
    inputs = tokenizer(prompt, return_tensors="pt").to(device)

    # Generate response
    with torch.no_grad():
        output = model.generate(**inputs, max_length=150, pad_token_id=tokenizer.eos_token_id)

    # Decode response
    response = tokenizer.decode(output[0], skip_special_tokens=True)

    # Extract only the assistant's response
    if "[ASSISTANT]" in response:
        response = response.split("[ASSISTANT]")[-1].strip()  # Remove everything before [ASSISTANT]

    print("Bargenix Bot:", response)  # Print chatbot response


🤖 Bargaining Chatbot is ready! Type 'exit' to quit.
You: give me discount
Bargenix Bot: I don’t
You: why?
Bargenix Bot: here’s a code for you: H


KeyboardInterrupt: Interrupted by user

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# Load the trained model
MODEL_PATH = "/content/bargenix-lora-model"  # Replace with your trained model path
device = "cuda" if torch.cuda.is_available() else "cpu"

tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
model = AutoModelForCausalLM.from_pretrained(MODEL_PATH, torch_dtype=torch.float16, device_map="auto").to(device)

# Define system instructions


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
import torch
import re  # Import regex module for number detection

# Define product details
product_name = "Premium Cotton Shirt"
original_price = 500  # Original price in currency
stock = 10  # Items in stock

# System prompt with product context and relevancy enforcement
system_prompt = (
    f"[SYSTEM] You are Bargenix, an AI that negotiates product prices. "
    f"Your only job is to bargain the price of '{product_name}', currently priced at {original_price}, with {stock} in stock. "
    f"Ignore irrelevant topics and only respond to messages related to the product, pricing, or negotiation. "
    f"Politely reject any unrelated queries.\n"
)

print(f"🤖 Bargaining Chatbot is ready! ({product_name} - {original_price} available: {stock})")
print("Type 'exit' to quit.")

while True:
    user_message = input("You: ")  # Get user input
    if user_message.lower() == "exit":
        print("Bargenix Bot: Goodbye! 👋")
        break  # Exit the loop

    # Check if the message is relevant (contains negotiation keywords or numbers)
    relevant_keywords = ["price", "discount", "offer", "deal", "bargain", "negotiate", "cost"]
    has_number = bool(re.search(r'\d+', user_message))  # Detects any number in the message

    if not any(keyword in user_message.lower() for keyword in relevant_keywords) and not has_number:
        print("Bargenix Bot: I only discuss pricing and negotiation for this product. Let's talk about the deal! 🤝")
        continue  # Skip processing and wait for a relevant message

    # Format user input with system instructions and product context
    prompt = f"{system_prompt}[USER] {user_message}\n[ASSISTANT]"

    # Tokenize input
    inputs = tokenizer(prompt, return_tensors="pt").to(device)

    # Generate response
    with torch.no_grad():
        output = model.generate(**inputs, max_length=150, pad_token_id=tokenizer.eos_token_id)

    # Decode response
    response = tokenizer.decode(output[0], skip_special_tokens=True)

    # Extract only the assistant's response
    if "[ASSISTANT]" in response:
        response = response.split("[ASSISTANT]")[-1].strip()  # Remove everything before [ASSISTANT]

    print("Bargenix Bot:", response)  # Print chatbot response



🤖 Bargaining Chatbot is ready! (Premium Cotton Shirt - 500 available: 10)
Type 'exit' to quit.
You: discount?
Bargenix Bot: Alright. I’ll do 400. But only if you buy now
You: 300?
Bargenix Bot: Hmm… I like your spirit. How about 230? But only if you buy


KeyboardInterrupt: Interrupted by user

In [ ]:
import torch
import re
from transformers import AutoModelForCausalLM, AutoTokenizer

# Load the trained model
MODEL_PATH = "/content/bargenix-lora-model"  # Replace with actual trained model path
device = "cuda" if torch.cuda.is_available() else "cpu"

tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
model = AutoModelForCausalLM.from_pretrained(MODEL_PATH, torch_dtype=torch.float16, device_map="auto").to(device)

# Define product details
product_name = "Premium Cotton Shirt"
original_price = 500  # Price in currency
stock = 10  # Available stock

# System prompt with bargaining logic
system_prompt = (
    f"[SYSTEM] You are Bargenix, an AI that negotiates product prices. "
    f"Your job is to bargain the price of '{product_name}', priced at {original_price}, with {stock} in stock. "
    f"Offer discounts only when necessary, negotiate hard, and refuse unrealistic prices. "
    f"Ignore irrelevant topics and politely redirect to price negotiation.\n"
)

print(f"🤖 Bargaining Chatbot is ready! ({product_name} - {original_price} available: {stock})")
print("Type 'exit' to quit.")

while True:
    user_message = input("You: ")  # Get user input
    if user_message.lower() == "exit":
        print("Bargenix Bot: Goodbye! 👋")
        break  # Exit the loop

    # Check if the message is relevant (contains negotiation keywords or numbers)
    relevant_keywords = ["price", "discount", "offer", "deal", "bargain", "negotiate", "cost"]
    has_number = bool(re.search(r'\d+', user_message))  # Detects any number in the message

    if not any(keyword in user_message.lower() for keyword in relevant_keywords) and not has_number:
        print("Bargenix Bot: I only discuss pricing and negotiation for this product. Let's talk about the deal! 🤝")
        continue  # Skip processing and wait for a relevant message

    # Format user input with system instructions and product context
    prompt = f"{system_prompt}[USER] {user_message}\n[ASSISTANT]"

    # Tokenize input
    inputs = tokenizer(prompt, return_tensors="pt").to(device)

    # Generate response using the trained model
    with torch.no_grad():
        output = model.generate(**inputs, max_length=150, pad_token_id=tokenizer.eos_token_id)

    # Decode response
    response = tokenizer.decode(output[0], skip_special_tokens=True)

    # Extract only the assistant's response
    if "[ASSISTANT]" in response:
        response = response.split("[ASSISTANT]")[-1].strip()  # Remove everything before [ASSISTANT]

    print("Bargenix Bot:", response)  # Print chatbot response


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

ERROR:asyncio:Task exception was never retrieved
future: <Task finished name='Task-106' coro=<Server.serve() done, defined at /usr/local/lib/python3.11/dist-packages/uvicorn/server.py:68> exception=SystemExit(1)>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/uvicorn/server.py", line 163, in startup
    server = await loop.create_server(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/asyncio/base_events.py", line 1536, in create_server
    raise OSError(err.errno, msg) from None
OSError: [Errno 98] error while attempting to bind on address ('0.0.0.0', 8000): address already in use

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-99-032cd558cd61>", line 52, in <cell line: 0>
    uvicorn.run(app, h

🤖 Bargaining Chatbot is ready! (Premium Cotton Shirt - 500 available: 10)
Type 'exit' to quit.
Bargenix Bot: I only discuss pricing and negotiation for this product. Let's talk about the deal! 🤝
Bargenix Bot: I only discuss pricing and negotiation for this product. Let's talk about the deal! 🤝
Bargenix Bot: Hi there! You’ve got an eye for our premium cotton shirt. How about 400? But I think 500 is more appropriate. How about 450? How about 425? Hmm… I like your spirit
Bargenix Bot: Hmm… I like your spirit. How about 337? But only if
